In [1]:
!pip install git+https://github.com/UN-GCPDS/python-gcpds.utils.git

  Cloning https://github.com/UN-GCPDS/python-gcpds.utils.git to /tmp/pip-req-build-7qrk6gcs
  Running command git clone -q https://github.com/UN-GCPDS/python-gcpds.utils.git /tmp/pip-req-build-7qrk6gcs
  Created wheel for gcpds-utils: filename=gcpds_utils-0.1a3-py3-none-any.whl size=96942 sha256=efc3ac6afccd0e7abab4c92b8dba3a69a77d8da8b04d4d60cdd9d12b7d56742c
  Stored in directory: /tmp/pip-ephem-wheel-cache-fylr4mos/wheels/27/94/14/3c2a3e71cf1df95b4a5b2174f21f469d678f59507e1d72f207
Successfully built gcpds-utils
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
!pip install mne==0.19

     |████████████████████████████████| 6.4 MB 5.0 MB/s 
  Attempting uninstall: mne
    Found existing installation: mne 0.22.0
    Uninstalling mne-0.22.0:
      Successfully uninstalled mne-0.22.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
import sys
sys.path.insert(0,'/kaggle/input/mi-eeg-classmeth/')

In [5]:
from gcpds.utils import loaddb
import sys
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
from joblib import dump, load
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split,GridSearchCV, StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score, pairwise_distances
from mne.viz import plot_topomap
from MIfunctions import * #importar todas las funciones de manejo de eeg con mne.
import os
import time
from sklearn.metrics import make_scorer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import pickle

In [6]:
tau =  4
overlap=1-0.5
ti = 4
tf = 40
tti = np.arange(ti,tf-tau+(tau*overlap),tau*overlap)
ttf = np.arange(ti+tau,tf+tau*overlap,tau*overlap)
f_frec = np.array([tti,ttf]).T
#vtw= np.array([[0,2],[1,3],[2,4],[3,5],[4,6],[5,7]])
tau =  1
overlap=1-0.75
ti = 2
tf = 6
tti = np.arange(ti,tf-tau+(tau*overlap),tau*overlap)
ttf = np.arange(ti+tau,tf+tau*overlap,tau*overlap)
vtw = np.array([tti,ttf]).T
#vtw= np.array([[0,2]])
#f_frec = np.array([[2.5,4.5]])
subjects = np.arange(52)+1
subjects=np.delete(subjects,[28,33])
for sbj in subjects[30:35]:#subjects:
    db = loaddb.GIGA('../input/giga-science-gcpds/GIGA_MI_ME/')
    db.load_subject(sbj)
    print(f'The subject {sbj} has {db.runs} runs')
    Xraw, y = db.get_data(reject_bad_trials=True,classes=['left hand mi', 'right hand mi'])
    #run[i], class_[i] = db.get_run(i,classes=['left hand', 'right hand'])
    Xraw = np.asarray(Xraw)
    Xraw=Xraw[:,:,::2]
    print(Xraw.shape)
    y = np.asarray(y,dtype=np.float64).ravel()
    y[y==0]=-1
    print(y.shape)
    fs =db.metadata['sampling_rate']/2
    channels_names = db.metadata['channel_names']
    montage = mne.channels.read_montage(db.metadata['montage'], channels_names)
    info = mne.create_info(channels_names, sfreq=fs, ch_types="eeg",
                            montage=montage) 
    feat = FB_feats(fs=fs,vtw=vtw,f_frec=f_frec,gammad=1,feat='kcon')
    xxx = feat.fit_transform(Xraw,y) 
    #opt = tf.keras.optimizers.RMSprop(learning_rate=1e-1)
    steps=[('CSP_window_time',Window_band_CSP_eppoch(fs=fs,vtw=vtw,f_frec=f_frec,PCov=True)),
            ('flat',flatt()),
            ('cla', LDA())]    
    method = Pipeline(steps,memory='/kaggle/tmp/datospipeline')
    parameters ={'CSP_window_time__ncomp':[6]}
    scores = {'acc': 'accuracy','kappa': make_scorer(cohen_kappa_score,greater_is_better=True)}    
    cv = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
    grid_search = GridSearchCV(method, parameters,cv=cv,verbose=10,
                               scoring=scores,refit='acc',n_jobs=-1)
    #grid_search = GridSearchCV(method, parameters,cv=cv,verbose=10,
     #                          scoring=kappa_,n_jobs=-1)
    #try:
    grid_search.fit(xxx,y)
    results = grid_search.cv_results_
    filters = grid_search.best_estimator_['CSP_window_time'].filters
    ncomp = grid_search.best_estimator_['CSP_window_time'].ncomp
    #%% topoplots
    savedata ={
        'steps':steps,
        'f_frec':f_frec,
        'vtw':vtw,
        'best_param':grid_search.best_params_,       
        'cv_results':results
    }
    #dump(savedata,'sujeto_'+str(sbj)+'_cka_featuresCSP_BCI2a_acc.joblib')
    dump(savedata,'tau_'+str(tau)+'_sujeto_'+str(sbj)+'_elastic_featuresCSP_GIGA_acc.joblib')
    print('sbj'+str(sbj)+'acc'+str(np.round(grid_search.best_score_*100,4)))

The subject 32 has 5 runs
(200, 64, 1792)
(200,)
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  1.1min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  1.2min remaining:   29.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.5min finished


sbj32acc49.75
The subject 33 has 5 runs
(199, 64, 1792)
(199,)
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  1.0min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  1.1min remaining:   27.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.4min finished


sbj33acc56.75
The subject 35 has 5 runs
(200, 64, 1792)
(200,)
Fitting 10 folds for each of 1 candidates, totalling 10 fits


...

sbj35acc84.5
The subject 36 has 5 runs
(195, 64, 1792)
(195,)
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  1.0min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  1.0min remaining:   26.6s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.4min finished


sbj36acc52.5641
The subject 37 has 5 runs
(200, 64, 1792)
(200,)
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  1.0min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  1.1min remaining:   27.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.4min finished


sbj37acc69.75


In [7]:
!zip Results-joblib.zip ./*.joblib
#!zip Results-images.zip ./*.eps

  adding: tau_1_sujeto_32_elastic_featuresCSP_GIGA_acc.joblib (deflated 62%)
  adding: tau_1_sujeto_33_elastic_featuresCSP_GIGA_acc.joblib (deflated 63%)
  adding: tau_1_sujeto_35_elastic_featuresCSP_GIGA_acc.joblib (deflated 63%)
  adding: tau_1_sujeto_36_elastic_featuresCSP_GIGA_acc.joblib (deflated 60%)
  adding: tau_1_sujeto_37_elastic_featuresCSP_GIGA_acc.joblib (deflated 63%)
